Docker pure python 3.11 build (DataSphere 1 GPU A100 • 28 vCPUs • 119 ГБ RAM1 GPU A100 • 28 vCPUs • 119 ГБ RAM)
```
FROM ubuntu:22.04
ENV DEBIAN_FRONTEND=noninteractive

RUN apt-get update && apt-get install -y --no-install-recommends \
      ca-certificates \
      python3.11 python3.11-venv python3.11-distutils \
      python3-pip \
    && ln -sf /usr/bin/python3.11 /usr/bin/python3 \
    && ln -sf /usr/bin/pip3 /usr/bin/pip \
    && python3 -m pip install -U pip \
    && rm -rf /var/lib/apt/lists/*

RUN useradd -ms /bin/bash --uid 1000 jupyter
USER jupyter
WORKDIR /home/jupyter
```

In [1]:
!python3 --version

Python 3.11.0rc1


In [2]:
%pip install murmurhash==1.0.13
%pip install numpy==2.1.2
%pip install tensorboard==2.20.0
%pip install transformers==4.50.3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
%pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import json
from functools import partial

import torch
from torch.utils.data import DataLoader

from modeling import utils
from modeling.dataloader import BatchProcessor
from modeling.dataset import Dataset
from modeling.loss import IdentityLoss
from modeling.metric import NDCGSemanticMetric, RecallSemanticMetric
from modeling.models import TigerModel, CorrectItemsLogitsProcessor
from modeling.trainer import Trainer
from modeling.utils import parse_args, create_logger, fix_random_seed

/home/jupyter/.local/lib/python3.11/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/kernel/lib/python3.11/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/kernel/lib/python3.11/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?t

In [5]:
def parse_config(config_path):
    with open(config_path) as f:
        params = json.load(f)
    
    return params

In [6]:
# config = parse_config('configs/tiger_kmeans_train_config.json')
config = parse_config('configs/tiger_tuned_kmeans_train_config.json')
print('Training config: \n{}'.format(json.dumps(config, indent=2)))

Training config: 
{
  "experiment_name": "tuned_tiger_beauty_kmeans",
  "dataset": {
    "inter_json_path": "../data/Beauty/inter.json",
    "index_json_path": "../data/Beauty/tuned_index_rqkmeans.json",
    "num_codebooks": 4,
    "max_sequence_length": 20,
    "sampler_type": "tiger"
  },
  "dataloader": {
    "train_batch_size": 256,
    "validation_batch_size": 256
  },
  "model": {
    "embedding_dim": 128,
    "codebook_size": 256,
    "num_positions": 20,
    "user_ids_count": 2000,
    "num_heads": 6,
    "num_encoder_layers": 4,
    "num_decoder_layers": 4,
    "dim_feedforward": 1024,
    "d_kv": 64,
    "dropout": 0.1,
    "activation": "relu",
    "num_beams": 100,
    "top_k": 20,
    "layer_norm_eps": 1e-06,
    "initializer_range": 0.02
  },
  "optimizer": {
    "lr": 0.0003
  },
  "train_epochs_num": 20
}


In [7]:
SEED_VALUE = 42
fix_random_seed(SEED_VALUE)

In [8]:
print('Current DEVICE: {}'.format(utils.DEVICE))

Current DEVICE: cuda


In [9]:
dataset = Dataset.create(
    inter_json_path=config['dataset']['inter_json_path'],
    max_sequence_length=config['dataset']['max_sequence_length'],
    sampler_type=config['dataset']['sampler_type'],
    is_extended=True
)
train_sampler, validation_sampler, test_sampler = dataset.get_samplers()
num_codebooks = config['dataset']['num_codebooks']
user_ids_count = config['model']['user_ids_count']

[2025-10-27 14:02:15] [INFO]: Train dataset size: 131413
[2025-10-27 14:02:15] [INFO]: Validation dataset size: 22363
[2025-10-27 14:02:15] [INFO]: Test dataset size: 22363
[2025-10-27 14:02:15] [INFO]: Max item id: 12100


In [10]:
batch_processor = BatchProcessor.create(
    config['dataset']['index_json_path'], num_codebooks, user_ids_count
)

train_dataloader = DataLoader(
    dataset=train_sampler,
    batch_size=config['dataloader']['train_batch_size'],
    drop_last=True,
    shuffle=True,
    collate_fn=batch_processor
)

validation_dataloader = DataLoader(
    dataset=validation_sampler,
    batch_size=config['dataloader']['validation_batch_size'],
    drop_last=False,
    shuffle=False,
    collate_fn=batch_processor
)

eval_dataloader = DataLoader(
    dataset=test_sampler,
    batch_size=config['dataloader']['validation_batch_size'],
    drop_last=False,
    shuffle=False,
    collate_fn=batch_processor
)

model = TigerModel(
    embedding_dim=config['model']['embedding_dim'],
    codebook_size=config['model']['codebook_size'],
    sem_id_len=num_codebooks,
    user_ids_count=user_ids_count,
    num_positions=config['model']['num_positions'],
    num_heads=config['model']['num_heads'],
    num_encoder_layers=config['model']['num_encoder_layers'],
    num_decoder_layers=config['model']['num_decoder_layers'],
    dim_feedforward=config['model']['dim_feedforward'],
    num_beams=config['model']['num_beams'],
    num_return_sequences=config['model']['top_k'],
    activation=config['model']['activation'],
    d_kv=config['model']['d_kv'],
    dropout=config['model']['dropout'],
    layer_norm_eps=config['model']['layer_norm_eps'],
    initializer_range=config['model']['initializer_range'],
    logits_processor=partial(
        CorrectItemsLogitsProcessor,
        config['dataset']['num_codebooks'],
        config['model']['codebook_size'],
        config['dataset']['index_json_path'],
        config['model']['num_beams']
    )
).to(utils.DEVICE)

In [11]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Overall parameters: {total_params:,}')
print(f'Trainable parameters: {trainable_params:,}')

Overall parameters: 5,236,352
Trainable parameters: 5,236,352


In [12]:
loss_function = IdentityLoss(
    predictions_prefix='loss',
    output_prefix='loss'
)  # Passes through the loss computed inside the model without modification

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['optimizer']['lr'],
)

codebook_size = config['model']['codebook_size']
ranking_metrics = {
    'ndcg@5': NDCGSemanticMetric(5, codebook_size, num_codebooks),
    'ndcg@10': NDCGSemanticMetric(10, codebook_size, num_codebooks),
    'ndcg@20': NDCGSemanticMetric(20, codebook_size, num_codebooks),
    'recall@5': RecallSemanticMetric(5, codebook_size, num_codebooks),
    'recall@10': RecallSemanticMetric(10, codebook_size, num_codebooks),
    'recall@20': RecallSemanticMetric(20, codebook_size, num_codebooks)
}

print('Everything is ready for training process!')

Everything is ready for training process!


In [13]:
config.get('train_epochs_num')

20

In [ ]:
trainer = Trainer(
    experiment_name=config['experiment_name'],
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    eval_dataloader=eval_dataloader,
    model=model,
    optimizer=optimizer,
    loss_function=loss_function,
    ranking_metrics=ranking_metrics,
    epoch_cnt=config.get('train_epochs_num'),
    step_cnt=config.get('train_steps_num'),
    best_metric='ndcg@20',
    epochs_threshold=config.get('early_stopping_threshold', 40),
    valid_step=256,
    eval_step=256
)

best_checkpoint = trainer.train()
trainer.save()

print('Training finished!')

[2025-10-27 14:02:31] [DEBUG]: Start training...
[2025-10-27 14:02:31] [DEBUG]: Start epoch 0
[2025-10-27 14:02:34] [DEBUG]: Running validation on step 0...
[2025-10-27 14:04:01] [DEBUG]: Running validation on step 0 is done!
[2025-10-27 14:04:01] [DEBUG]: Running eval on step 0...
[2025-10-27 14:05:27] [DEBUG]: Running eval on step 0 is done!


0 0.0010496710470791757 0 0.0008125865693457602


[2025-10-27 14:05:44] [DEBUG]: Running validation on step 256...
[2025-10-27 14:07:11] [DEBUG]: Running validation on step 256 is done!
[2025-10-27 14:07:11] [DEBUG]: Running eval on step 256...
[2025-10-27 14:08:36] [DEBUG]: Running eval on step 256 is done!


256 0.004906057839033691 0.0010496710470791757 0.004069395047823091


[2025-10-27 14:08:53] [DEBUG]: Running validation on step 512...
[2025-10-27 14:10:19] [DEBUG]: Running validation on step 512 is done!
[2025-10-27 14:10:19] [DEBUG]: Running eval on step 512...
[2025-10-27 14:11:45] [DEBUG]: Running eval on step 512 is done!
[2025-10-27 14:11:45] [DEBUG]: Start epoch 1


...

3072 0.042905849957063126 0.041729748115121416 0.03621697930054988


[2025-10-27 14:43:34] [DEBUG]: Start epoch 6
[2025-10-27 14:43:50] [DEBUG]: Running validation on step 3328...
[2025-10-27 14:45:17] [DEBUG]: Running validation on step 3328 is done!
[2025-10-27 14:45:17] [DEBUG]: Running eval on step 3328...
[2025-10-27 14:46:44] [DEBUG]: Running eval on step 3328 is done!


3328 0.04372398984129439 0.042905849957063126 0.03827312872282205


[2025-10-27 14:47:01] [DEBUG]: Running validation on step 3584...
[2025-10-27 14:48:28] [DEBUG]: Running validation on step 3584 is done!
[2025-10-27 14:48:28] [DEBUG]: Running eval on step 3584...
[2025-10-27 14:49:55] [DEBUG]: Running eval on step 3584 is done!
[2025-10-27 14:49:56] [DEBUG]: Start epoch 7
[2025-10-27 14:50:13] [DEBUG]: Running validation on step 3840...
[2025-10-27 14:51:39] [DEBUG]: Running validation on step 3840 is done!
[2025-10-27 14:51:39] [DEBUG]: Running eval on step 3840...
[2025-10-27 14:53:06] [DEBUG]: Running eval on step 3840 is done!


3840 0.04449970371148638 0.04372398984129439 0.03899691105275388


In [ ]:
print('Final evaluation is being performed...')

trainer.load(best_checkpoint)
trainer.eval()

Eval from file checkpoint

In [15]:
from pathlib import Path

checkpoint_path = Path(f"../checkpoints/{config['experiment_name']}_final_state.pth")
state_dict = torch.load(checkpoint_path, map_location=utils.DEVICE)

trainer.load(state_dict)
trainer.eval()

/tmp/ipykernel_23294/3603050627.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_path, map_location=utils.DEVICE)
[2025-10-25 09:51:13]

ndcg@5 0.022584518243720333
ndcg@10 0.028540282975111846
ndcg@20 0.03560548419336809
recall@5 0.035147341591020884
recall@10 0.05374949693690471
recall@20 0.08187631355363771
